In [19]:
import sqlite3
from sqlite3 import Error
import pandas as pd
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('sqlite3 version: ', sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query10 = """
        SELECT f.facid,
                f.name AS facility,
                (f.guestcost * g.guest_slots + f.membercost * m.member_slots) - (initialoutlay + 3*monthlymaintenance) AS total_revenue
        FROM Facilities AS f
        LEFT JOIN (
            SELECT facid, SUM(slots) AS member_slots
            FROM Bookings
            WHERE memid <> 0
            GROUP BY facid
            ) AS m
        USING (facid)
        LEFT JOIN (
            SELECT facid, SUM(slots) AS guest_slots
            FROM Bookings
            WHERE memid = 0
            GROUP BY facid
            ) AS g
        USING (facid)
        WHERE (f.guestcost * g.guest_slots + f.membercost * m.member_slots) - (initialoutlay + 3*monthlymaintenance) < 1000
        ORDER BY total_revenue DESC
        """
    cur.execute(query10)
 
    rows10 = cur.fetchall()
 
    df10 = pd.DataFrame(rows10)
    print(df10)
    print('\n')
    #for row in rows10:
    #    print(row)

#----------------
    query11 = """
        SELECT m.surname, m.firstname, s.surname AS recommenders_surname, s.firstname AS recommenders_firstname
        FROM Members AS m
        LEFT JOIN (
            SELECT memid, surname, firstname
            FROM Members
            WHERE memid <>0
        ) AS s ON m.recommendedby = s.memid
        WHERE recommendedby <> ' '
        ORDER BY m.surname, m.firstname
    """
    cur.execute(query11)
 
    rows11 = cur.fetchall()
 
    df11 = pd.DataFrame(rows11)
    print(df11)
    print('\n')
    #for row in rows11:
    #    print(row)
        
#-------------------
    query12 = """
        SELECT memid, surname, firstname, facid, facility, COUNT( facility ) as usage_rate
        FROM (
            SELECT b.bookid, b.facid, b.memid, f.name AS facility, m.surname, m.firstname
            FROM Bookings AS b
            LEFT JOIN Members AS m
                USING ( memid )
            LEFT JOIN Facilities AS f
                USING ( facid )
            WHERE memid <>0
        ) AS s
        GROUP BY memid, facility
        ORDER BY memid, facid
    """
    cur.execute(query12)
 
    rows12 = cur.fetchall()
 
    df12 = pd.DataFrame(rows12)
    print(df12)
    print('\n')
    #for row in rows12:
    #    print(row)

#------------
    query13 = """
        SELECT month, facid, facility, COUNT(facility) AS usage_rate
        FROM (
            SELECT f.name as facility, b.facid, strftime('%m', b.starttime) AS month
            FROM Bookings as b
            LEFT JOIN Facilities as f
                USING (facid)
        ) AS s
        GROUP BY month, facility
        ORDER BY month, facid
    """
    cur.execute(query13)
 
    rows13 = cur.fetchall()
    
    df13 = pd.DataFrame(rows13)
    print(df13)
    print('\n')
    #for row in rows13:
    #    print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

sqlite3 version:  2.6.0
Query all tasks
   0                1       2
0  3     Table Tennis  -170.0
1  8       Pool Table  -175.0
2  7    Snooker Table  -255.0
3  2  Badminton Court -2243.5


                    0          1         2          3
0               Bader   Florence  Stibbons     Ponder
1               Baker       Anne  Stibbons     Ponder
2               Baker    Timothy   Farrell     Jemima
3              Boothe        Tim    Rownam        Tim
4             Butters     Gerald     Smith     Darren
5              Coplin       Joan     Baker    Timothy
6             Crumpet      Erica     Smith      Tracy
7                Dare      Nancy  Joplette     Janice
8             Farrell      David      None       None
9             Farrell     Jemima      None       None
10              GUEST      GUEST      None       None
11            Genting    Matthew   Butters     Gerald
12               Hunt       John   Purview  Millicent
13              Jones      David  Joplette     Janic